In [45]:
!pip install geopandas shapely

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point 
import os

In [47]:
# get rid of NA values and sort by species name
df = pd.read_csv("alldata.csv")
df = df.dropna(subset=['Observation_ID'])
df = df.drop(df.columns[0], axis=1)
df = df.sort_values(by='Species_name')
df.head()

,Observation_ID,Date_of_observation,User_id,User_Tree_id,Species_name,Lat,Long,State_name,Leaves_fresh,Leaves_mature,Leaves_old,Flowers_bud,Flowers_open,Flowers_male,Flowers_Female,Fruits_unripe,Fruits_ripe,Fruits_open
377464,467438.0,2020-10-13,22219.0,94805.0,Aabehayat Mango- Mangifera indica,9.78619,76.44449,Kerala,2.0,2.0,1.0,0.0,0.0,-2.0,-2.0,0.0,0.0,-2.0
402822,489809.0,2021-03-02,22219.0,94803.0,Aabehayat Mango- Mangifera indica,9.78499,76.44959,Kerala,1.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
433873,544927.0,2021-11-23,22219.0,94805.0,Aabehayat Mango- Mangifera indica,9.78619,76.44449,Kerala,1.0,2.0,1.0,0.0,0.0,-2.0,-2.0,0.0,0.0,-2.0
433874,544928.0,2021-11-24,22219.0,94805.0,Aabehayat Mango- Mangifera indica,9.78619,76.44449,Kerala,1.0,2.0,1.0,1.0,1.0,-2.0,-2.0,0.0,0.0,-2.0
402823,489810.0,2021-03-09,22219.0,94803.0,Aabehayat Mango- Mangifera indica,9.78499,76.44959,Kerala,1.0,2.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0


In [48]:
states_shapefile = gpd.read_file("india/gadm41_IND_3.shp")

In [6]:
def find_indian_state(latitude, longitude, gdf):
    point = Point(longitude, latitude)
    
    for _, state in gdf.iterrows():
        if state['geometry'].contains(point):
            return state['NAME_1']
    return None

state = find_indian_state(13.07248, 80.24340, states_shapefile)
print(state) # IF THE FUNCTION WORKS THIS SHOULD OUTPUT TAMIL NADU

Tamil Nadu


In [7]:
# NOW WE ITERATE THROUGH THE ROWS AND IF THE STATE NAME IS NAN, THEN IT IS REPLACED BY THE STATE NAME
# OUTPUTTED FROM THE FUNCTION

for i, row in df.iterrows():
    if pd.isna(row["State_name"]):
        df.at[i, "State_name"] = find_indian_state(row["Lat"], row["Long"], states_shapefile)

/opt/anaconda3/envs/cs589/lib/python3.10/site-packages/shapely/predicates.py:526: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


In [ ]:
# Save updated all_data.csv
df.to_csv('updated_alldata.csv', index=False) # SAVED IT

In [ ]:
# export new state dataframes to citizenData folder
new_dir_path = 'citizenData'
try:
    # Create the directory
    os.mkdir(new_dir_path)
    print(f"Directory '{new_dir_path}' created.")
except FileExistsError:
    print(f"Directory '{new_dir_path}' already exists.")

for state_name in df["State_name"].unique(): # This piece of code creates a new df for each state with Year and Week formatting to match the reference data
    state_df = df[df["State_name"] == state_name]
    state_df["Date_of_observation"] = pd.to_datetime(state_df["Date_of_observation"], format='mixed')
    state_df["Year"] = state_df["Date_of_observation"].dt.isocalendar().year
    state_df["Week"] = state_df["Date_of_observation"].dt.dayofyear // 7.583 #364/48 (dt.dayofyear gives an index, thus 364)
    state_df["Week"] = state_df["Week"].astype(int)
    
    state_df.to_csv(f'citizenData/{state_name}_citizenData.csv', index=False)

In [11]:
# load lookup dicts for id -> name and name -> id from species_codes.csv
species_codes = pd.read_csv("species codes.csv", encoding='unicode_escape')

species_id_to_name = {}
species_name_to_id = {}

for i, row in species_codes.iterrows():
    species_id_to_name[row["species_id"]] = "{}-{}".format(row["species_primary_common_name"], row["species_scientific_name"])
    species_name_to_id["{}-{}".format(row["species_primary_common_name"], row["species_scientific_name"]).lower().replace(" ", "")] = row["species_id"]

In [12]:
# paths to citizen and reference data. create if they do not exist
citizen_path = "all data/citizen"
reference_path = "all data/reference"

try:
    # Create the nested directories
    os.makedirs(citizen_path)
    print(f"Nested directories '{citizen_path}' created.")
except FileExistsError:
    print(f"Nested directories '{citizen_path}' already exists.")

try:
    # Create the nested directories
    os.makedirs(reference_path)
    print(f"Nested directories '{reference_path}' created.")
except FileExistsError:
    print(f"Nested directories '{reference_path}' already exists.")

Nested directories 'all data/citizen' already exists.
Nested directories 'all data/reference' already exists.


In [56]:
# concatenate data for andaman and nicobar and delhi
aani_df = pd.read_csv("citizenData/Andaman and Nicobar Islands_citizenData.csv")
aan_df = pd.read_csv("citizenData/Andaman and Nicobar_citizenData.csv")
pd.concat([aan_df, aani_df],ignore_index=True).to_csv('citizenData/Andaman and Nicobar Islands_citizenData.csv',index=False)

nct_delhi_df = pd.read_csv("citizenData/NCT of Delhi_citizenData.csv")
delhi_df = pd.read_csv("citizenData/Delhi_citizenData.csv")
pd.concat([delhi_df, nct_delhi_df],ignore_index=True).to_csv('citizenData/Delhi_citizenData.csv')

In [13]:
# update names in reference data

citizen_states = {}
reference_states = {}

for root, _, files in os.walk("citizenData"):
    for file_name in files:
        if file_name != ".DS_Store":
            file_path = os.path.join(root, file_name)
            # Read the CSV file into a DataFrame

            df = pd.read_csv(file_path)

            state = file_name.replace("_citizenData.csv", "")
            state = state.replace(" ", "_").lower()
            
            citizen_states[state] = file_name

for root, _, files in os.walk("reference_data"):
    for file_name in files:
        if file_name != ".DS_Store":
            file_path = os.path.join(root, file_name)
            
            df = pd.read_csv(file_path)
            
            state = file_name.replace("pvt_", "")
            state = state.replace(".csv", "")
            state = state.replace(" ", "_").lower()
            
            if state == "maharastra":
                reference_states["maharashtra"] = file_name
            else:
                reference_states[state] = file_name
                
species_codes = pd.read_csv("species codes.csv", encoding='unicode_escape')

species_id_to_name = {}
species_name_to_id = {}

for i, row in species_codes.iterrows():
    species_id_to_name[row["species_id"]] = "{}-{}".format(row["species_primary_common_name"], row["species_scientific_name"])
    species_name_to_id["{}-{}".format(row["species_primary_common_name"], row["species_scientific_name"]).lower().replace(" ", "")] = row["species_id"]
    
for state in reference_states:
    ref_df = pd.read_csv(f"reference_data/{reference_states[state]}")
    # remove unnamed columns if they exist
    for col in ref_df.columns:
        if 'Unnamed' in col or col == 'id':
            ref_df = ref_df.drop(col, axis=1)
    ref_df['species_name'] = ref_df['species_id'].map(species_id_to_name)
    ref_df.to_csv(f"all data/reference/{state}.csv", index=False)

In [102]:
# update names in citizen data
def reformat_mango_string(input_string):
    index = input_string.find("mango")
    
    prefix = input_string[:index]
    suffix = input_string[index+5:]  

    output_string = f"mango:{prefix}{suffix}"
    
    return output_string

for state in citizen_states:
    cit_df = pd.read_csv(f"citizenData/{citizen_states[state]}")
    print(state)

    for i, row in cit_df.iterrows():
        name = row["Species_name"].lower().replace(" ", "")
        if name == "arjuntree-terminaliaarjuna":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1083]
            continue
        if name == "axlewoodtree-anogeissuslatifolia":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1009]
            continue
        if name == "chiku-sapodilla-manilkarazapota\xa0":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1188]
            continue
        if name == "dyer'soleander-wrightiatinctoria":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1181]
            continue
        if name == "ficusmollis-softfig":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1197]
            continue
        if name == "frangipani-templetree-plumeriarubra":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1176]
            continue
        if name == "garuga-kharpat-garugapinnata":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1038]
            continue
        if name == "ghostrree-sterculiaurens":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1078]
            continue
        if name == "indianfrankincense-boswelliaserrata":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1195]
            continue
        if name == "indiancoraltree-erythrinaindica":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1034]
            continue
        if name == "kadamba-neolamarckiacadamba":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1058]
            continue
        if name == "lanneacoromandelica-indianashtree":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1194]
            continue
        if name == "mexicanoleander-yellowoleander-cascabelathevetia":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1177]
            continue
        if name == "nightfloweringjasmine-harsingar-nyctanthesarbor-tristis":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1059]
            continue
        if name == "prosopiscineraria-khejri":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1201]
            continue
        if name == "raintree-samaneasaman":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1162]
            continue
        if name == "redsilk-cotton-bombaxceiba":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1015]
            continue
        if name == "whitesilk-cotton-ceibapentandra":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1021]
            continue
        if name == "yellow-silkcottontree-cochlospermumreligiosum":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1023]
            continue
        if name == "karkat-dogteak-dilleniapentagyna":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1032]
            continue
        if name == "chosamango-mangiferaindica":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1108]
            continue
        if name == "falsewhiteteak-mallotusnudiflorus":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1088]
            continue
        if name == "floss-silktree-ceibaspeciosa":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1022]
            continue
        if name == "largesebesten-bairola-cordiawallichii":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1026]
            continue
        if name == "roxburghskydia-pulia-kydiacalycina":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1044]
            continue
        if name == "wildrose-rosawebbiana":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1206]
            continue
        if name == "albiziaodoratissima-blacksiris":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1199]
            continue
        if name == "anogeissuspendula-kardhai":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1198]
            continue
        if name == "brokenbonestree-oroxylumindicum":
            cit_df.loc[i, "Species_name"] = "Broken Bones Tree-Oroxylum Indicum"
            continue
        if name == "pyinmatree-andamancrapemyrtle-lagerstroemiahypoleuca":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1216]
            continue
        if name == "crataevareligiosa-garlic-peartree":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1196]
            continue
        if name == "guh-de-three-leafcapertree-cratevaadansonii":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1217]
            continue
        if name == "aabehayatmango-mangiferaindica":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1091]
            continue
        if name == "bedu-punjabfig-ficuspalmata":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1235]
            continue
        if name == "chinar-platanusorientalis":
            cit_df.loc[i, "Species_name"] = "Chinar-Platanus Orientalis"
            continue
        if name == "prunusnepalensis-sohiong":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1192]
            continue
        if name == "tecomellaundulata-roheda":
            cit_df.loc[i, "Species_name"] = species_id_to_name[1200]
            continue
        if name == "tigersmilkspruce-falconeriainsignis":
            cit_df.loc[i, "Species_name"] = "Tiger's Milk Spruce-Falconeria Insignis"
            continue
        if "mango" in name:
            mango_string = reformat_mango_string(name.lower().replace(" ", ""))
            if mango_string == "mango:(allvarieties)-mangiferaindica":
                cit_df.loc[i, "Species_name"] = species_id_to_name[1090]
            else :
                cit_df.loc[i, "Species_name"] = species_id_to_name[species_name_to_id[mango_string]]
            continue
        if species_name_to_id[name]:
            cit_df.loc[i, "Species_name"] = species_id_to_name[species_name_to_id[name]]
        
    cit_df.to_csv(f"all data/citizen/{state}.csv")

{'whitebabool-acacialeucophloea': 1000, 'babool-acacianilotica': 1001, 'woodapple-aeglemarmelos': 1002, 'indianhorsechestnut-aesculusindica': 1003, 'treeofheaven-ailanthusexcelsa': 1004, 'krishnasiris-albiziaamara': 1005, 'siris-albizialebbeck': 1006, 'whitesiris-albiziaprocera': 1007, "devil'stree-alstoniascholaris": 1008, 'axlewood-anogeissuslatifolia': 1009, 'greymangrove-avicenniamarina': 1010, 'neem-azadirachtaindica': 1011, 'purplebauhinia-bauhiniapurpurea': 1012, 'jhinjheri-bauhiniaracemosa': 1013, 'semla-bauhiniaretusa': 1014, 'redsilkcotton-bombaxceiba': 1015, 'toddypalm-borassusflabellifer': 1016, 'flameoftheforest-buteamonosperma': 1017, 'blackdammer-canariumstrictum': 1018, 'fish-tailpalm-caryotaurens': 1019, 'indianlaburnum-cassiafistula': 1020, 'whitesilkcotton-ceibapentandra': 1021, 'flosssilktree-ceibaspeciosa': 1022, 'yellowsilkcotton-cochlospermumreligiosum': 1023, 'mountaincoffee-coffeaarabica': 1024, 'robustacoffee-coffearobusta': 1025, 'largesebesten-cordiawallichi

andhra_pradesh
goa
uttarakhand
haryana
chandigarh
sikkim
madhya_pradesh
jharkhand
himachal_pradesh
chhattisgarh
delhi
bihar
punjab
andaman_and_nicobar_islands
dadra_and_nagar_haveli
assam
kerala
puducherry
rajasthan
gujarat
lakshadweep
arunachal_pradesh
jammu_and_kashmir
uttar_pradesh
manipur
meghalaya
west_bengal
karnataka
tamil_nadu
nagaland
tripura
maharashtra
odisha


In [66]:
# Make sure all attributes have been added to citizen data

for state in citizen_states:
    if state == 'nct_of_delhi' or state == 'andaman_and_nicobar': # these got merged into multiple files
        continue
    cit_df = pd.read_csv(f"citizenData/{citizen_states[state]}")
    other_cit_df = pd.read_csv(f"all data/citizen/{state}.csv")
    other_cit_df['Lat'] = cit_df['Lat']
    other_cit_df['Long'] = cit_df['Long']
    other_cit_df['Observation_ID'] = cit_df['Observation_ID']
    other_cit_df['User_id'] = cit_df['User_id']
    other_cit_df['User_Tree_id'] = cit_df['User_Tree_id']
    other_cit_df = other_cit_df.drop('Unnamed: 0', axis=1)
    other_cit_df = other_cit_df.drop('Unnamed: 0.1', axis=1)
    other_cit_df.to_csv(f'all data/citizen/{state}.csv', index=False)